In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import janitor
import warnings
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line,Funnel
from pyecharts.charts import Bar, Timeline
from pyecharts.commons.utils import JsCode
from mpl_toolkits.mplot3d import Axes3D

#解决pyecharts图表空白问题
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
CurrentConfig.ONLINE_HOST = 'https://assets.pyecharts.org/assets/'
#解决中文字体不显示，正负号不显示问题
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

%matplotlib widget


warnings.filterwarnings('ignore')

In [21]:
df = pd.read_excel('./附件.xlsx')

In [22]:
df.drop_duplicates(inplace=True)

In [203]:
df.dtypes

顾客编号      int64
大类编码      int64
大类名称     object
中类编码      int64
中类名称     object
小类编码      int64
小类名称     object
销售日期      int64
销售月份      int64
商品编码     object
规格型号     object
商品类型     object
单位       object
销售数量    float64
销售金额    float64
商品单价    float64
是否促销     object
dtype: object

In [204]:
df.head(1)

,顾客编号,大类编码,大类名称,中类编码,中类名称,小类编码,小类名称,销售日期,销售月份,商品编码,规格型号,商品类型,单位,销售数量,销售金额,商品单价,是否促销
0,0,12,蔬果,1201,蔬菜,120109,其它蔬菜,20150101,201501,DW-1201090311,,生鲜,个,8.0,4.0,2.0,否


In [23]:
data = (
    df
    [df['销售数量']>=0]
    [df['销售金额']>=0]
    [df['销售日期']!=20150229]
    [~df['销售数量'].isnull()]
    .assign(销售日期=lambda d:pd.to_datetime(d['销售日期'].astype('string')))  
    [['顾客编号','大类名称','销售日期','销售数量','销售金额','商品类型']]
)

In [9]:
data.shape#(42721, 5)

(42721, 5)

# 商品维度分析

## 日销售规律

In [ ]:
#一天同一个客户的所有购买记录算作一个购物篮（一个销售单）

In [10]:
data_day = (
    data
    .groupby(['销售日期','顾客编号'])
    .agg({'销售数量':'sum','销售金额':'sum'})
    .reset_index()
    .groupby(['销售日期'])
    .agg({'销售数量':'sum','销售金额':'sum','顾客编号':'count'})
    .rename(columns = {'顾客编号':'购物篮数量'})

)

In [11]:
data_day

,销售数量,销售金额,购物篮数量
销售日期,,,
2015-01-01,339.666,2870.73,64
2015-01-02,427.572,3558.75,69
2015-01-03,317.010,2531.34,64
2015-01-04,372.073,2624.11,74
2015-01-05,366.401,3195.59,75
...,...,...,...
2015-04-26,583.925,4465.21,91
2015-04-27,366.354,2553.21,73
2015-04-28,334.049,2657.64,66


## 周销售规律探索

In [16]:
data_week = (
    data
    .groupby(['销售日期','顾客编号'])
    .agg({'销售数量':'sum','销售金额':'sum'})
    .reset_index()
    .groupby(['销售日期'])
    .agg({'销售数量':'sum','销售金额':'sum','顾客编号':'count'})
    .reset_index()
    .pipe(lambda d:d[~d['销售日期'].isin(['2015-01-01','2015-2-11','2015-2-12','2015-2-13','2015-2-14','2015-2-15','2015-2-16'
                   ,'2015-2-17','2015-2-18','2015-2-19','2015-2-20','2015-2-21','2015-2-22','2015-2-23'
                   ,'2015-2-24','2015-3-5','2015-3-8','2015-4-5'])])
    .assign(week=lambda d:d['销售日期'].dt.dayofweek+1)
    .groupby('week')
    .sum()

)

In [17]:
data_week

,销售数量,销售金额,顾客编号
week,,,
1,8111.491,69820.00,1353
2,6933.363,75084.63,1308
3,10549.968,94216.05,1771
4,5807.440,47544.87,1093
5,6411.577,55500.95,1149
6,6547.637,53613.82,1121
7,7083.308,59988.38,1148


In [110]:
data_day.sum()#['购物篮数量']

销售数量      51444.784
销售金额     455768.700
购物篮数量      8943.000
dtype: float64

## 人气商品分析

In [44]:
(
    data
    [['顾客编号','销售日期','大类名称','销售数量']]
    .groupby(['顾客编号','销售日期'],as_index=False)
    .apply(lambda d:d.assign(含指定大类的销售数量=lambda d:d['销售数量'].sum()))
    .assign(含指定大类的销售单数=1)
    .groupby(['顾客编号','销售日期','大类名称'],as_index=False)
    .agg({'含指定大类的销售数量':'mean','含指定大类的销售单数':'mean'})
    .groupby('大类名称')
    .agg({'含指定大类的销售数量':'sum','含指定大类的销售单数':'sum'})
    .assign(购物篮系数=lambda d:d['含指定大类的销售数量']/d['含指定大类的销售单数'])
    .assign(人气指数=lambda d:d['含指定大类的销售数量']/data_day.sum()['购物篮数量'])
    .sort_values(by='人气指数',ascending=False)
   
)

,含指定大类的销售数量,含指定大类的销售单数,购物篮系数,人气指数
大类名称,,,,
蔬果,28976.358,5167.0,5.607966,3.240116
日配,27891.235,3330.0,8.375746,3.118778
休闲,24072.721,3345.0,7.196628,2.691795
粮油,22289.395,3044.0,7.322403,2.492385
洗化,10705.415,1581.0,6.771293,1.197072
酒饮,10503.154,1160.0,9.054443,1.174455
肉禽,5875.648,1018.0,5.771756,0.657011
熟食,4514.506,733.0,6.158944,0.504809
冲调,3737.658,420.0,8.899186,0.417942


## 商品关联度分析
#支持度、可信度、提升度
* 支持度：同时包含A和B的交易/总交易*100%

   支持度代表这组关联商品的份额是否足够大
* 可信度：同时包含A和B的交易/包含A的总交易*100%

   购买A同时购买B的概率，关联程度的强弱，
* 提升度：可信度/商品B在总交易中出现的概率

   该关联规则是否有利用价值 


### 支持度

In [163]:
#自定义函数求各大类商品之间的支持度
def zhichidu(data):
    dl_list = list(data['大类名称'].unique())
    d = {}
    for i in dl_list:
        l1 = []
        for j in dl_list:
            df = ( data
                     .groupby(['销售日期','顾客编号'])
                    [['大类名称']]
                    .sum()
                    )
            if i!=j :
                n = round((df['大类名称'].str.contains('%s\S*%s|%s\S*%s'%(i,j,j,i),regex=True).sum())
                          /(df['大类名称'].count()),4
                         )
            else:
                n=round((df['大类名称'].str.contains(i).sum())
                          /(df['大类名称'].count()),2
                         )
            l1.append(n)
        d[i]=l1
    d_f = pd.DataFrame(d,index=dl_list)
    return d_f 

In [189]:
zhichi = zhichidu(data)
zhichi.where(zhichi>=0.03)#.style.format('{:.2%}')#.background_gradient(vmin=0.03)#

,蔬果,粮油,日配,洗化,熟食,休闲,肉禽,家居,针织,烘焙,冲调,文体,水产,酒饮,家电
蔬果,0.5800,0.1912,0.2115,0.0731,0.0617,0.1853,0.0945,NaN,NaN,NaN,NaN,NaN,NaN,0.0458,NaN
粮油,0.1912,0.3400,0.1278,0.0653,NaN,0.1299,0.0353,NaN,NaN,NaN,NaN,NaN,NaN,0.0381,NaN
日配,0.2115,0.1278,0.3700,0.0573,NaN,0.1514,0.0363,NaN,NaN,NaN,NaN,NaN,NaN,0.0473,NaN
洗化,0.0731,0.0653,0.0573,0.1800,NaN,0.0718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
熟食,0.0617,NaN,NaN,NaN,0.0800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
休闲,0.1853,0.1299,0.1514,0.0718,NaN,0.3700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0657,NaN
肉禽,0.0945,0.0353,0.0363,NaN,NaN,NaN,0.1100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
家居,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
针织,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN
烘焙,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
支持度高于3%的商品大类有：蔬果，粮油，日配，洗化，熟食，休闲，肉禽，酒饮

### 可信度

In [184]:
#自定义函数求各大类商品之间的可信度
#同时包含A和B的交易/包含A的总交易*100%
def kexindu(data):
    dl_list = list(data['大类名称'].unique())
    d = {}
    for i in dl_list:
        l1 = []
        for j in dl_list:
            df = ( data
                     .groupby(['销售日期','顾客编号'])
                    [['大类名称']]
                    .sum()
                    )
            if i!=j :
                n = round((df['大类名称'].str.contains('%s\S*%s|%s\S*%s'%(i,j,j,i),regex=True).sum())
                          /(df['大类名称'].str.contains(i).sum()),2
                         )
            else:
                n=1
            l1.append(n)
        d[i]=l1
    d_f = pd.DataFrame(d,index=dl_list)
    return d_f

In [190]:
kexindu(data).where(kexindu(data)>=0.3)#.style.format('{:.2%}')

,蔬果,粮油,日配,洗化,熟食,休闲,肉禽,家居,针织,烘焙,冲调,文体,水产,酒饮,家电
蔬果,1.00,0.56,0.57,0.41,0.75,0.50,0.83,0.44,0.45,0.61,0.37,0.48,0.83,0.35,0.33
粮油,0.33,1.00,0.34,0.37,0.33,0.35,0.31,0.40,0.32,NaN,0.39,0.42,0.36,NaN,NaN
日配,0.37,0.38,1.00,0.32,0.36,0.40,0.32,0.33,NaN,NaN,0.47,0.38,0.30,0.36,0.35
洗化,NaN,NaN,NaN,1.00,NaN,NaN,NaN,0.39,0.32,NaN,NaN,NaN,NaN,NaN,0.37
熟食,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
休闲,0.32,0.38,0.41,0.41,0.30,1.00,NaN,0.37,0.39,0.39,0.52,0.49,NaN,0.51,0.33
肉禽,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
家居,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
针织,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN
烘焙,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN


In [ ]:
#购买任何大类的商品的同时会购买的的可信度在3%以上的商品是蔬果，其次是粮油、日配、休闲。说明蔬果、粮油、日配、休闲是超市客流量的主要来源

### 提升度

In [177]:
#自定义函数求各大类商品之间的提升度
#提升度：可信度/商品B在总交易中出现的概率
def tishengdu(data):
    dl_list = list(data['大类名称'].unique())
    d = {}
    for i in dl_list:
        l1 = []
        for j in dl_list:
            df = ( data
                     .groupby(['销售日期','顾客编号'])
                    [['大类名称']]
                    .sum()
                    )
            if i!=j :
                n = (df['大类名称'].str.contains('%s\S*%s|%s\S*%s'%(i,j,j,i),regex=True).sum())/(df['大类名称'].str.contains(i).sum())
                m = round(n/((df['大类名称'].str.contains(j).sum())/(df['大类名称'].count())),2)
            else:
                n = np.nan
                m = round(n/((df['大类名称'].str.contains(j).sum())/(df['大类名称'].count())),2)
            l1.append(m)
        d[i]=l1
    d_f = pd.DataFrame(d,index=dl_list)
    return d_f 

In [192]:
tisheng = tishengdu(data)
tisheng.style.background_gradient(vmin=1.1,cmap='YlOrRd')

,蔬果,粮油,日配,洗化,熟食,休闲,肉禽,家居,针织,烘焙,冲调,文体,水产,酒饮,家电
蔬果,nan,0.970000,0.980000,0.720000,1.300000,0.860000,1.440000,0.760000,0.780000,1.060000,0.650000,0.840000,1.440000,0.610000,0.560000
粮油,0.970000,nan,1.010000,1.090000,0.970000,1.020000,0.910000,1.180000,0.930000,0.820000,1.150000,1.230000,1.060000,0.860000,0.770000
日配,0.980000,1.010000,nan,0.870000,0.970000,1.090000,0.860000,0.880000,0.770000,0.600000,1.250000,1.010000,0.800000,0.980000,0.930000
洗化,0.720000,1.090000,0.870000,nan,0.590000,1.090000,0.520000,2.190000,1.790000,1.260000,1.090000,1.480000,0.700000,0.990000,2.090000
熟食,1.300000,0.970000,0.970000,0.590000,nan,0.810000,1.610000,0.430000,0.430000,1.360000,0.440000,0.710000,1.450000,0.830000,0.530000
休闲,0.860000,1.020000,1.090000,1.090000,0.810000,nan,0.630000,0.990000,1.040000,1.040000,1.380000,1.320000,0.740000,1.360000,0.870000
肉禽,1.440000,0.910000,0.860000,0.520000,1.610000,0.630000,nan,0.510000,0.500000,0.490000,0.500000,0.410000,2.130000,0.480000,0.760000
家居,0.760000,1.180000,0.880000,2.190000,0.430000,0.990000,0.510000,nan,3.670000,1.260000,1.130000,2.630000,1.280000,0.960000,2.950000
针织,0.780000,0.930000,0.770000,1.790000,0.430000,1.040000,0.500000,3.670000,nan,0.000000,1.210000,3.190000,0.220000,0.610000,1.710000
烘焙,1.060000,0.820000,0.600000,1.260000,1.360000,1.040000,0.490000,1.260000,0.000000,nan,0.000000,0.000000,2.810000,0.430000,0.000000


In [196]:
#满足支持度高于3%的商品大类有：蔬果，粮油，日配，洗化，熟食，休闲，肉禽，酒饮
#以上大类中，支持度较高的有
(
    tisheng
    .filter(items=['蔬果','粮油','日配','洗化','熟食','休闲','肉禽','酒饮'])
    .filter(items=['蔬果','粮油','日配','洗化','熟食','休闲','肉禽','酒饮'],axis=0)
    .pipe(lambda d:d.where(d>1.1))
)

,蔬果,粮油,日配,洗化,熟食,休闲,肉禽,酒饮
蔬果,NaN,NaN,NaN,NaN,1.30,NaN,1.44,NaN
粮油,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
日配,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
洗化,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
熟食,1.30,NaN,NaN,NaN,NaN,NaN,1.61,NaN
休闲,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.36
肉禽,1.44,NaN,NaN,NaN,1.61,NaN,NaN,NaN
酒饮,NaN,NaN,NaN,NaN,NaN,1.36,NaN,NaN


### 小节
#从结果可以看出蔬果和熟食，蔬果和肉禽，熟食和肉禽，休闲和酒饮之间不论是相互的提升度比较高。可以在绑定销售或者，货架陈列在一起

# RFM模型分析

In [65]:
(
    data
    .groupby(['顾客编号','销售日期'],as_index=False)
    .agg({'销售日期':'max','销售金额':'sum'})
    .assign(消费次数=1)
    .groupby('顾客编号',as_index=False)
    .agg({'销售日期':'max','销售金额':'sum','消费次数':'count'})
    .assign(R=lambda d:(d['销售日期'].max()-d['销售日期']).apply(lambda x:x.days))
    .assign(F=lambda d:d['消费次数'])
    .assign(M=lambda d:d['销售金额'])
    [['顾客编号','R','F','M']]
    .pipe(lambda d:d.case_when(  ((d['R']<d['R'].mean())&(d['F']>=d['F'].mean())&(d['M']>=d['M'].mean())),'222重要价值用户',
                                 ((d['R']<d['R'].mean())&(d['F']>=d['F'].mean())&(d['M']<d['M'].mean())),'221一般价值用户',
                                 ((d['R']<d['R'].mean())&(d['F']<d['F'].mean())&(d['M']>=d['M'].mean())),'212重要发展用户',
                                 ((d['R']<d['R'].mean())&(d['F']<d['F'].mean())&(d['M']<d['M'].mean())),'211一般发展用户',
                                 ((d['R']>=d['R'].mean())&(d['F']>=d['F'].mean())&(d['M']>=d['M'].mean())),'122重要保持用户',
                                 ((d['R']>=d['R'].mean())&(d['F']>=d['F'].mean())&(d['M']<d['M'].mean())),'121一般保持用户',
                                 ((d['R']>=d['R'].mean())&(d['F']<d['F'].mean())&(d['M']>=d['M'].mean())),'112重要挽留用户',
                                 ((d['R']>=d['R'].mean())&(d['F']<d['F'].mean())&(d['M']<d['M'].mean())),'111一般挽留用户','0',
                                     column_name='客户分类'
                                    ))
    .groupby('客户分类')
    .agg({'顾客编号':'count','R':'mean','F':'mean','M':'mean'})

)

,顾客编号,R,F,M
客户分类,,,,
111一般挽留用户,950,83.714737,1.296842,61.198874
112重要挽留用户,128,74.703125,1.804688,311.394063
121一般保持用户,31,69.483871,4.483871,121.900968
122重要保持用户,46,65.804348,5.934783,681.328478
211一般发展用户,761,20.954008,1.522996,62.735782
212重要发展用户,101,19.851485,2.306931,282.703762
221一般价值用户,127,14.519685,5.047244,123.660315
222重要价值用户,468,11.209402,10.758547,492.844915


2612

In [210]:
(
    pd.pivot_table(data[['销售日期','顾客编号','大类名称','销售数量','销售金额']],
                   index=['销售日期','顾客编号'],columns=['大类名称'],values='销售数量',
                   aggfunc=np.sum,fill_value=0
                  )

)

大类名称              休闲  冲调  家居  家电  文体     日配   水产  洗化   烘焙   熟食   粮油     肉禽  \
销售日期       顾客编号                                                              
2015-01-01 0     0.0   0   0   0   0  0.000  0.0   0  0.0  4.0  0.0  0.000   
           1     0.0   0   0   0   0  0.000  0.0   0  0.0  0.0  7.0  0.000   
           2     0.0   0   0   0   0  1.000  0.0   0  0.0  0.0  0.0  0.000   
           3     0.0   0   0   0   0  1.000  0.0   0  0.0  0.0  0.0  0.000   
           4     0.0   0   0   0   0  1.862  0.0   0  0.0  0.0  0.0  0.000   
...              ...  ..  ..  ..  ..    ...  ...  ..  ...  ...  ...    ...   
2015-04-30 2607  0.0   0   0   0   0  0.000  0.0   0  0.0  0.0  1.0  0.000   
           2608  0.0   0   0   0   0  0.770  0.0   0  0.0  0.0  0.0  1.136   
           2609  1.0   0   0   0   0  1.000  0.0   0  0.0  0.0  0.0  0.000   
           2610  0.0   0   0   0   0  0.000  0.0   0  0.0  0.0  0.0  0.000   
           2611  0.0   0   0   0   0  1.000  0.0   0  0.0  0.0  0.0  0.000   

大类名称                蔬果   酒饮  针织  
销售日期       顾客编号                  
2015-01-01 0     9.908  0.0   0  
           1     1.366  0.0   0  
           2     6.745  0.0   0  
           3     1.748  0.0   0  
           4     0.686  4.0   0  
...                ...  ...  ..  
2015-04-30 2607  1.121  0.0   0  
           2608  0.000  0.0   0  
           2609  0.000  9.0   0  
           2610  0.000  2.0   0  
           2611  0.000  0.0   0  

[8943 rows x 15 columns]